In [ ]:
import json
train_path = "sample_data.json" # from https://github.com/urchade/GLiNER/blob/main/examples/sample_data.json

with open(train_path, "r") as f:
    data = json.load(f)

In [ ]:
# available models: https://huggingface.co/urchade
import torch
from gliner import GLiNER, GlinerTrainer
 
model = GLiNER.from_pretrained("urchade/gliner_small")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model = model.to(device)

In [ ]:
eval_data = {
    "entity_types": ["Person", 'Award'],
    "samples": data[:50]
}

trainer = GlinerTrainer(model, 
                        train_data = data, 
                        batch_size = 4,
                        grad_accum_every = 16,
                        lr_encoder = 1e-5,
                        lr_others = 5e-5, 
                        freeze_token_rep = False, 
                        val_every_step = 1000, 
                        val_data = eval_data,
                        checkpoint_every_epoch = 15, # Or checkpoint_every_step if you use steps
                        max_types=25,
                        
                        #Uncomment these if you want to train using fp16 
                        #optimizer_kwargs = { "eps": 1e-7},  # Using higher eps might cause NaN loss
                        #accelerator_kwargs = {"mixed_precision" : "fp16" },
)


trainer.train(num_epochs=50) # Or by steps: trainer.train(num_steps=50)

In [ ]:
trainer.model.save_pretrained("small")

In [ ]:
md = GLiNER.from_pretrained("small", local_files_only=True)

In [ ]:
text = """
Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards,[note 3] a record three UEFA Men's Player of the Year Awards, and four European Golden Shoes, the most by a European player. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (128) and international appearances (205). He is one of the few players to have made over 1,200 professional career appearances, the most by an outfield player, and has scored over 850 official senior career goals for club and country, making him the top goalscorer of all time.
"""

# Labels for entity prediction
labels = ["Person", "Award"] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])